# Parallel Processing for Street View Analysis

## Overview
This notebook implements parallel processing for analyzing thousands of Street View images using LangGraph with multiple concurrent nodes.

## Problem Statement
- **Scale**: Thousands of locations × multiple orientations (horizon/ground/sky) = 10k+ images
- **Bottleneck**: Sequential AI model calls would take hours
- **Solution**: Parallel processing with bounded concurrency

## Architecture
- **Input**: CSV with Street View URLs (from `google_apis.ipynb`)
- **Processing**: LangGraph parallel nodes for vision analysis
- **Output**: Graded/analyzed results per image

## Concurrency Strategy
- **Measure**: Single call latency (L) for vision model
- **Calculate**: k ≈ T × L (where T = target throughput)
- **Start**: k=16 nodes, scale up based on provider limits
- **Monitor**: 429 errors, latency percentiles, success rates

## Key Considerations
- **Provider Limits**: API rate limits and concurrent request caps
- **Rate Limiting**: Client-side throttling to avoid 429s
- **Retry Logic**: Exponential backoff with jitter
- **Checkpointing**: Idempotent tasks to avoid duplicate billing
- **Budget Guards**: Max requests per minute/hour

## Implementation Plan
1. Load CSV and create work queue
2. Implement bounded concurrency with LangGraph
3. Add retry logic and rate limiting
4. Monitor metrics and scale accordingly

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().absolute().parent))  # This makes the parent directory available so you can use clean absolute imports like from src.graph import ...

In [ ]:
from src.state import MultiState
from src.make_graph import multimodal_node
from langgraph.graph import StateGraph, START, END

def build_parallel_graph(checkpointer, n_nodes=2,save_display=False) -> StateGraph:
    """
    Get the builder for the graph
    """
    builder = StateGraph(MultiState)
    # nodes
    for i in range(n_nodes):
        builder.add_node(f"multimodal_agent_{i}", multimodal_node)
    # edges
    # Need to investigate more on parallel edges: do i need to join? Do i use `Send`?
    builder.add_edge(START, "get_data")
    for i in range(n_nodes):
        builder.add_edge(f"get_data", f"multimodal_agent_{i}")
        builder.add_edge(f"multimodal_agent_{i}", "join")
    builder.add_edge("join", END)

    graph = builder.compile(checkpointer=checkpointer)

    if save_display:
        # save the graph display to file
        img = graph.get_graph().draw_mermaid_png() # returns bytes
        # save the bytes to file 
        with open("./graph.png", "wb") as f:
            f.write(img)
        print("Graph display saved to ./src/graph.png")

    return graph

Using OPENAI model
